In [2]:
from langchain_groq import  ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
EXCHANGE_API= os.environ.get('EXCHANGE_API')

In [4]:
#Create the tool
@tool
def multiply (a:int,b:int)-> int:
    """Given two number a and b this tool their product"""
    return a*b

In [5]:
multiply.invoke({"a":3,"b":5})

15

In [6]:
# Bind tool
llm = ChatGroq(model="llama-3.3-70b-versatile")
llmBind=llm.bind_tools([multiply])

In [7]:

llmBind.invoke("Hi,How are You")

AIMessage(content="I'm doing well, thanks for asking. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 231, 'total_tokens': 256, 'completion_time': 0.090909091, 'prompt_time': 0.021747376, 'queue_time': 0.20452411799999998, 'total_time': 0.112656467}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b64ae9e-ea9a-4f56-b030-304a69bccca4-0', usage_metadata={'input_tokens': 231, 'output_tokens': 25, 'total_tokens': 256})

In [8]:
#Tool calling
res =llmBind.invoke("Can you multiply 3 and 4")

In [9]:
res.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 4},
 'id': 'call_9arq',
 'type': 'tool_call'}

In [10]:
# Tool execution
multiply.invoke({'name': 'multiply',
 'args': {'a': 3, 'b': 4},
 'id': 'call_pd84',
 'type': 'tool_call'})

ToolMessage(content='12', name='multiply', tool_call_id='call_pd84')

In [11]:
query =HumanMessage("What is product of 4 and 54")
message=[query]

In [12]:
llmBind.invoke(message)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tzrp', 'function': {'arguments': '{"a": 4, "b": 54}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 235, 'total_tokens': 254, 'completion_time': 0.069090909, 'prompt_time': 0.014265597, 'queue_time': 0.20415618300000002, 'total_time': 0.083356506}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-888c8430-de96-484e-9d93-3fe5cdaec146-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 54}, 'id': 'call_tzrp', 'type': 'tool_call'}], usage_metadata={'input_tokens': 235, 'output_tokens': 19, 'total_tokens': 254})

# Currancy converter

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_API}/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [29]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'PKR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747526401,
 'time_last_update_utc': 'Sun, 18 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1747612801,
 'time_next_update_utc': 'Mon, 19 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 281.795}

In [30]:
convert.invoke({'base_currency_value':10, 'conversion_rate':281.795})

2817.9500000000003

In [31]:
tools=llm.bind_tools([get_conversion_factor,convert])

In [39]:
messages = [HumanMessage('What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR')]

In [40]:
ai_message =tools.invoke(messages)

In [41]:
messages.append(ai_message)

In [42]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [43]:
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6rw', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_kqgd', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 349, 'total_tokens': 387, 'completion_time': 0.138181818, 'prompt_time': 0.029433189, 'queue_time': 0.21971488, 'total_time': 0.167615007}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-218751d4-633f-4301-ba0b-d5f2496d9ee5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'PKR'

In [44]:
tools.invoke(messages).content

'The conversion factor between USD and PKR is 281.795. Based on this conversion factor, 10 USD is equal to 2817.95 PKR.'